# ====================================
# Jupyter Notebook: Function Calling 101
# ====================================


In [ ]:
"""
Scenario:
    We have 4 functions:
      1. get_weather_info(city)
      2. book_flight(loc_origin, loc_destination, datetime, airline)
      3. extract_entities
      4. tag text

We present them to the OpenAI model with 'function_descriptions'.
When the user asks a question that matches the function usage, 
the model will produce a structured function call (JSON) with name & arguments.

We'll manually parse the result from the model, call our Python function, 
and then feed that result back to the model to produce a final user-facing answer.

TODO:
  - Add more interesting scenarios (like file_complaint).
  - Experiment with 'function_call="auto"' vs. forced or required.
  - Combine multiple user requests into a single prompt to see if the model calls 
    multiple functions or just one.
"""

import os
import json
from datetime import datetime, timedelta
import openai


########################################################
# Section 1: Setup
########################################################

In [ ]:
"""
We'll use environment variables to store API keys.
Make sure you have an OPENAI_API_KEY environment variable set.
"""
from dotenv import load_dotenv

import openai
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
print("OpenAI Key Found:", bool(os.getenv("OPENAI_API_KEY")))


MODEL_NAME = "gpt-4o-mini"

# -------------------------------------------------
# 2) Define Python functions to be "called"
# -------------------------------------------------
In this cell, we define the **actual** Python functions the model can “call.”  
- `get_weather_info(city)`: Returns mocked weather data (in real usage, you'd call an actual weather API).  
- `book_flight(...)`: Pretends to book a flight and returns a JSON-formatted confirmation.

In [ ]:
def get_weather_info(city: str):
    """
    Dummy function that returns made-up weather data.
    In reality, you'd call a weather API (like OpenWeatherMap).
    """
    fake_data = {
        "Amsterdam": {"temp": 15, "condition": "Drizzle"},
        "New York": {"temp": 22, "condition": "Sunny"},
        "Paris": {"temp": 16, "condition": "Overcast"}
    }
    weather = fake_data.get(city, {"temp": 0, "condition": "Unknown"})
    
    return json.dumps({
        "city": city,
        "temperature_c": weather["temp"],
        "conditions": weather["condition"]
    })

def book_flight(loc_origin: str, loc_destination: str, datetime_str: str, airline: str):
    """
    Dummy function to 'book' a flight.
    In reality, you'd integrate with an airline or travel booking API.
    """
    return json.dumps({
        "status": "success",
        "origin": loc_origin,
        "destination": loc_destination,
        "datetime": datetime_str,
        "airline": airline,
        "confirmation_number": "ABC123XYZ"
    })

def extract_entities(text: str):
    """
    Dummy function that 'extracts' person names and ages from text.
    We'll simulate the result as a simple dictionary.
    In reality, you'd do more sophisticated NER or rely on LLM logic directly.
    """
    # Very naive "parser"
    # If it sees "Joe is 30" => we store that as an entity
    entities = []
    words = text.split()
    for i, w in enumerate(words):
        if w.lower() in ["joe", "mary", "bob"]:
            # check if next words might be "is <age>"
            if i+2 < len(words) and words[i+1].lower() in ["is"] and words[i+2].isdigit():
                entities.append({"name": w.capitalize(), "age": int(words[i+2])})
            else:
                entities.append({"name": w.capitalize(), "age": None})
    return json.dumps({"entities": entities})

def tag_text(text: str):
    """
    Dummy function for tagging text with sentiment + language.
    In reality, you'd call a sentiment classifier or language detection library.
    """
    # We'll simulate some trivial checks:
    sentiment = "neutral"
    if any(x in text.lower() for x in ["love", "great", "amazing"]):
        sentiment = "pos"
    elif any(x in text.lower() for x in ["hate", "terrible", "bad", "dislike"]):
        sentiment = "neg"

    # We'll do a naive language detection check for 'mi piace' => italian
    language = "en"
    if "mi piace" in text.lower():
        language = "it"
    
    return json.dumps({
        "sentiment": sentiment,
        "language": language
    })



# -------------------------------------------------
# 3) Describe these functions for OpenAI
# -------------------------------------------------
This cell simply displays all the functions (or “tools”) we’ve defined. We assign each function a name, description, and a JSON schema for its arguments, so the model knows how to call them.

In [ ]:
# List of function descriptions for API calls
function_descriptions = [
    {
        "type": "function",  # Defines this as a function tool
        "function": {
            "name": "get_weather_info",  # Function name used by the LLM
            "description": "Retrieve current weather information for a city.",
            "parameters": {
                "type": "object",  # Function expects an object as input
                "properties": {
                    "city": {
                        "type": "string",  # City name as input
                        "description": "City to retrieve weather data for, e.g., 'Amsterdam'."
                    }
                },
                "required": ["city"]  # City is mandatory
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "book_flight",
            "description": "Book a flight between two locations with a preferred airline.",
            "parameters": {
                "type": "object",
                "properties": {
                    "loc_origin": {  
                        "type": "string",  # Departure location
                        "description": "3-letter airport code or city name of departure."
                    },
                    "loc_destination": {
                        "type": "string",  # Arrival location
                        "description": "3-letter airport code or city name of arrival."
                    },
                    "datetime": {
                        "type": "string",  # Date/Time in ISO format
                        "description": "Flight date/time in ISO format, e.g., '2024-06-01 08:00'."
                    },
                    "airline": {
                        "type": "string",  # Airline preference
                        "description": "Preferred airline for booking."
                    }
                },
                "required": ["loc_origin", "loc_destination", "datetime", "airline"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_entities",
            "description": "Extract named entities (e.g., person name, age) from text.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",  # Input text containing entities
                        "description": "Text to analyze for named entity recognition (NER)."
                    }
                },
                "required": ["text"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "tag_text",
            "description": "Analyze text and tag it with sentiment and language.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",  # Input text to be classified
                        "description": "Text to be tagged with sentiment and language classification."
                    }
                },
                "required": ["text"]
            }
        }
    }
]

print("Function descriptions loaded.")  # Confirmation message


# 4) Define and Document the Function `test_call_model`

In this cell, we create a helper function named `test_call_model`. It:
1. Accepts a user prompt (`user_message`) and a `function_call` mode (`"auto"`, `"none"`, or `{"name": "...function_name..."}`).
2. Calls the OpenAI API with our **function (tool) descriptions** so the model knows which functions are available.
3. Returns the model's raw response, which may include a function call if it decides that is relevant.
python
Copy
Edit

 -------------------------------------------------
 Quick test with 'get_weather_info'
 -------------------------------------------------
 Purpose: 
 Test if the model calls the "get_weather_info" function 
 when we ask about the weather in Amsterdam.

In [ ]:
"""
We'll pass our function schema to the model. The model can decide
to call one function, multiple, or none, depending on user input.

- function_call="auto" => The model decides if/when to call.
- function_call="none" => The model cannot call any function.
- function_call={"name":"book_flight"} => Force it to call 'book_flight'.

Try toggling these below in the 'test_call_model' function.
"""

def test_call_model(user_message: str, function_call="auto"):
    """
    1) We send user_message + function_descriptions to the model
    2) We see if it returns a function call
    3) If so, we parse arguments, call the function ourselves
    4) Return final output
    """
    client = openai.OpenAI()

    completion = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": user_message}],
        tools=function_descriptions,  # 'functions' is now called 'tools'
        tool_choice=function_call,    # 'function_call' is now 'tool_choice'
    )
    response = completion.choices[0].message
    return response

## Quick Test with `get_weather_info`**
Here, we try a simple user prompt asking about the weather in Amsterdam.  
- We use `function_call="auto"` so the model may decide to invoke our `get_weather_info` function if it deems it relevant.  
- The cell prints out the raw model response so we can see if it includes a function call.

In [ ]:
# Let's do a quick test with something that calls 'get_weather_info'
user_prompt_1 = "What is the weather in Amsterdam?"
resp = test_call_model(user_prompt_1, function_call="auto")
print("Model response:\n", resp)
print("\nWe expect a 'function_call' to get_weather_info.")

# -------------------------------------------------
# 5) Parse the response and call the function
# -------------------------------------------------

 -------------------------------------------------
 Why This Code is Useful:
 Many LLM-based apps need to handle model outputs that
 specify a "function call." This code acts as a "bridge":
 - If the LLM wants to invoke a function (to fetch data or
   perform an action), we parse the model's tool_call info,
   call the real Python function, and then use another API
   call to produce a final, user-facing answer.
 - This approach is crucial for letting LLMs integrate
   with external systems (e.g., weather APIs, booking
   services) in a controlled, structured manner.


In [ ]:
# Initialize OpenAI client
client = openai.OpenAI()

def handle_function_call(response) -> str:
    """
    If the response contains a function call, execute it, return the result,
    and pass it back to the model for a final answer.
    """
    # If there's no function call, return the assistant's response as normal
    if response.content and not hasattr(response, "tool_calls"):
        return response.content

    # Extract function call information
    tool_calls = response.tool_calls
    if not tool_calls:
        return response.content

    # Process the first function call in the list
    tool_call = tool_calls[0]
    fn_name = tool_call.function.name
    fn_args = json.loads(tool_call.function.arguments)
    tool_call_id = tool_call.id  # Extract tool_call_id
    print(f"Model wants to call function {fn_name} with args {fn_args}")

    # Route to the correct function
    if fn_name == "get_weather_info":
        city_req = fn_args["city"]
        function_result = get_weather_info(city_req)

    elif fn_name == "book_flight":
        loc_origin = fn_args["loc_origin"]
        loc_dest = fn_args["loc_destination"]
        dt = fn_args["datetime"]
        airline = fn_args["airline"]
        function_result = book_flight(loc_origin, loc_dest, dt, airline)
    elif fn_name == "extract_entities":
        text = fn_args["text"]
        function_result = extract_entities(text)
    elif fn_name == "tag_text":
        text = fn_args["text"]
        function_result = tag_text(text)
    else:
        return "Function not recognized"

    # Ensure function_result is a string
    function_result = str(function_result)

    # Handle possible None content
    previous_content = f"Previously: {response.content}" if response.content else "Processing function call..."

    # Make a second model request, providing the function result
    # so it can finalize a user-facing response. 
    # This step is commonly used to incorporate the function's data
    # (e.g., "temp=15C, partly cloudy") into a full natural language
    # answer for the end user.
    second_response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "user", "content": previous_content},  
            {"role": "assistant", "tool_calls": response.tool_calls},  # Include the original tool call
            {
                "role": "tool",
                "tool_call_id": tool_call_id,
                "name": fn_name,
                "content": function_result
            }  
        ],
    )

    # Extract final answer
    final_answer = second_response.choices[0].message.content
    return final_answer

# Test it end-to-end
final_text = handle_function_call(resp)
print("\nFinal text back to user:\n", final_text)

# -------------------------------------------------
# 6) Another Example: Book a flight
# -------------------------------------------------
 Why This Is Useful:
 Demonstrates how a user request can trigger the "book_flight" function.
 This pattern can generalize to many use cases like "place an order",
 "schedule a meeting", etc. The LLM decides the best function to call
 and we do the behind-the-scenes work.

In [ ]:
"""
We'll ask a multi-part question that might trigger the model to call the second function.
"""

user_prompt_2 = "I want to book a flight from AMS to JFK on 2024-06-02 at 10:00 with Delta Airlines"
resp2 = test_call_model(user_prompt_2, function_call="auto")
print("Model response:\n", resp2)

final_text_2 = handle_function_call(resp2)
print("\nUser-Facing Answer:\n", final_text_2)


# -------------------------------------------------
# 7)  Section: Entity Extraction
# -------------------------------------------------
 Why This Is Useful:
 Shows how the LLM can parse unstructured text (e.g., "Joe is 30,
 Mary is older...") and call a function that extracts relevant entities
 or structured data. This is a building block for advanced data extraction,
 knowledge-base population, or record creation.

In [ ]:
"""
We have an 'extract_entities' function that looks for simple name/age pairs.
Let's see if the LLM picks that function for a user query describing people.
"""

user_prompt_3 = "Joe is 30, Mary is older but we don't know her age."
resp3 = test_call_model(user_prompt_3, function_call="auto")
print("\n=== Entity Extraction Test ===")
print("Model response:\n", resp3)
final_text_3 = handle_function_call(resp3)
print("\nUser-Facing Answer:\n", final_text_3)



# -------------------------------------------------
# 8) Section: Tagging
# -------------------------------------------------
 Why This Is Useful:
 Tagging or classification can be critical for sentiment analysis,
 content moderation, or language detection. By letting the LLM call
 a specialized "tag_text" function, you can unify your external logic
 (like a custom sentiment model) with the language reasoning of the LLM.

 (Implementation of the function or usage example would go here.)

In [ ]:
"""
We have a 'tag_text' function that returns a naive sentiment & language.
We'll see if the model calls it automatically if the user requests tagging.
"""

user_prompt_4 = "Can you tag this text for me: 'mi piace la pizza, it's amazing!'"
resp4 = test_call_model(user_prompt_4, function_call="auto")
print("\n=== Tagging Test ===")
print("Model response:\n", resp4)
final_text_4 = handle_function_call(resp4)
print("\nUser-Facing Answer:\n", final_text_4)

# -------------------------------------------------
# 9) Activity / Optional Challenge
# -------------------------------------------------

In [ ]:
"""
1) Try combining the two tasks into one user request:
   e.g. "What's the weather in New York, 
         and also please book me a flight from LAX to SFO next Friday at 9am with United."

   Observe if the model tries to call multiple functions or just one. 
   Because the standard function calling only returns ONE call at a time, 
   you might get partial coverage. 
   (HINT: you'll need to loop to handle multiple calls or do more advanced logic.)

2) Create a third function, e.g. "file_complaint(name, email, text)", 
   to simulate a user wanting to file a complaint about their flight.
   Add that to function_descriptions, 
   then see if the model picks it up when the user says 
   "I want to file a complaint about my missed flight. My name is Jane, email is jane@example.com"

3) Experiment with forcing a function call:
   - function_call="none": The model won't produce any function calls.
   - function_call={"name":"book_flight"}: The model *must* call the 'book_flight' function, 
     which might lead to it guessing arguments if the user didn't specify them.

4) If you want an advanced challenge, 
   handle repeated calls automatically:
   - If the model calls one function, you feed the result, 
     then it calls a second function, etc.
   - This is sometimes referred to as a "multi-step" or "agentic" approach.

Have fun and experiment!
"""


# -------------------------------------------------
# 10) End
# -------------------------------------------------